<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_33.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

In [9]:
from datetime import datetime

# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

current_year = datetime.now().year

# 2월 29일에 해당하는 행 선택
selected_rows = data[(data['일시'].dt.month == 2) & (data['일시'].dt.day == 29)]

# 선택한 행의 인덱스를 사용하여 삭제
data = data.drop(selected_rows.index)

In [10]:
# 계절 분류 함수 정의
def classify_season(month):
    if  month == 1 or month == 2 or month == 12:
        return '겨울'
    elif 3 <= month <= 5:
        return '봄'
    elif 6 <= month <= 8:
        return '여름'
    else:
        return '가을'

# '일시' 컬럼에서 월 정보를 추출하여 '계절' 열 추가
data['월'] = data['일시'].dt.month
data['계절'] = data['월'].apply(classify_season)

# 계절별 중앙값 계산
seasonal_medians = data.groupby('계절').transform('median')

# 결측값 대체
data.fillna(seasonal_medians, inplace=True)

# print(seasonal_medians)

<ipython-input-10-d4b59aa18d4d>:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seasonal_medians = data.groupby('계절').transform('median')


In [11]:
train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]
# 일시를 제외해보기로 함
target = data['평균기온']

In [12]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
poly.fit(train)
train_poly = poly.transform(train)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_poly, target)

# print(lr.score(train_poly, target))

from sklearn.metrics import mean_absolute_error
lr_pred = lr.predict(train_poly)

lr_mae = mean_absolute_error(target, lr_pred)

In [13]:
len(lr_pred)

22995

In [14]:
# 주어진 배열
input_array = np.array(lr_pred)  # 22995개의 행

# 365행을 갖는 62개의 열로 변환
output_array = input_array.reshape((63, 365))


# 각 열별로 중앙값 계산하여 365개의 행으로 만들기
medians = np.median(output_array, axis=0)
result_array = medians.reshape((365, 1))


submission['평균기온'] = result_array[:358]
print(submission.head(30))

            일시      평균기온
0   2023-01-01 -2.696393
1   2023-01-02 -1.657946
2   2023-01-03 -2.641253
3   2023-01-04 -3.213359
4   2023-01-05 -2.363843
5   2023-01-06 -2.018328
6   2023-01-07 -1.795802
7   2023-01-08 -2.532088
8   2023-01-09 -2.287620
9   2023-01-10 -1.565188
10  2023-01-11 -3.017676
11  2023-01-12 -2.956052
12  2023-01-13 -3.076480
13  2023-01-14 -3.390019
14  2023-01-15 -2.825668
15  2023-01-16 -2.788391
16  2023-01-17 -2.143644
17  2023-01-18 -2.952857
18  2023-01-19 -3.059613
19  2023-01-20 -2.339026
20  2023-01-21 -2.811074
21  2023-01-22 -2.397226
22  2023-01-23 -2.816248
23  2023-01-24 -2.700468
24  2023-01-25 -2.153491
25  2023-01-26 -2.549729
26  2023-01-27 -1.796992
27  2023-01-28 -2.053089
28  2023-01-29 -2.316952
29  2023-01-30 -3.113136


In [15]:
submission.to_csv("DEAGYUL_27_submission.csv", index=False)

print(submission.head())

           일시      평균기온
0  2023-01-01 -2.696393
1  2023-01-02 -1.657946
2  2023-01-03 -2.641253
3  2023-01-04 -3.213359
4  2023-01-05 -2.363843
